In [ ]:
#Deriving species labels from sorted USFWS camera trap imagery
#Requires MegaDetector sorted outputs in YOLO format

import os

# Specify the directory containing the YOLOv5 labels
labels_dir = '/path/to/labels/directory'

# Specify the directory to move non-animal labels
non_animal_dir = '/path/to/non-animal/labels/directory'

# Initialize an empty species mapping dictionary
species_mapping = {}

# Loop through each label file in the directory
for label_file in os.listdir(labels_dir):
    if label_file.endswith('.txt'):
        with open(os.path.join(labels_dir, label_file), 'r') as f:
            lines = f.readlines()

        modified_lines = []

        # Loop through each line in the label file
        for line in lines:
            class_id, x_center, y_center, width, height = map(float, line.split())

            # Extract the species name from the image path
            image_path = os.path.join(label_file.replace('.txt', '.jpg'))
            species_name = image_path.split('/')[3]  # Update the index as per your file structure

            # Check if the label is for the "Animal" class
            if int(class_id) == 0:
                # Check if the species name already exists in the mapping dictionary
                if species_name not in species_mapping:
                    # Generate a new class ID for the species and add it to the mapping dictionary
                    new_class_id = len(species_mapping)
                    species_mapping[species_name] = new_class_id
                else:
                    # Retrieve the existing class ID for the species from the mapping dictionary
                    new_class_id = species_mapping[species_name]
                
                # Modify the line with the updated class ID
                modified_line = f"{int(new_class_id)} {x_center} {y_center} {width} {height}"
                modified_lines.append(modified_line)
            else:
                # Move non-animal labels to the specified directory
                os.rename(
                    os.path.join(labels_dir, label_file),
                    os.path.join(non_animal_dir, label_file)
                )

        # Write the modified lines back to the label file
        with open(os.path.join(labels_dir, label_file), 'w') as f:
            f.write('\n'.join(modified_lines))

# Save the species mapping to a .txt file
species_mapping_file = '/path/to/species_mapping.txt'
with open(species_mapping_file, 'w') as f:
    for species_name, class_id in species_mapping.items():
        f.write(f"{species_name}: {class_id}\n")
